# Working with a database

This week's assignment has a few basic steps.  First, we're going to pull some data down off the internet and store it into our MySQL database.  Make sure that you use your username as part of the table name as show in the examples so that you don't create a problem for other students.

Then, we'll merge that with some data already in the database and calculate a few results.  When it comes to calculating the results, you can do so either with SQL or with Pandas operations.


## PART 1: Setup your database connection and table name

In the code below, change the value of the variable `MYTABLE` to use your own username rather that `'pboal'`

You can then use `MYTABLE` in the rest of your code to reference that table name.

In [2]:
#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!
# First, put in your user name below
#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!

MYTABLE='petersluu' + '_data'


# Then, create a connection to the same database, using the 
# same user name and password in the `mysql-examples.ipynb` 
# file.  Use conn as the database connection variable.
import os
import pymysql
from sqlalchemy import create_engine
import pandas as pd

host = 'slucor2020.cgdcoitnku0k.us-east-1.rds.amazonaws.com'
port = '3306'
user = 'slucor2020'
password = 'SLUcor2020'
database = 'hds5210'

conn = create_engine('mysql+pymysql://' + 
                     user + ':' + 
                     password + '@' + 
                     host + '/' + 
                     database, echo=False)


In [3]:
assert(MYTABLE != 'paulboal_data')
assert(conn.connect())

## PART 2: Bring in outside data

Grab the data from this URL and put it into a database table named with your `username_data`.

http://dhcs-chhsagency.opendata.arcgis.com/datasets/8e4f3a0c75b9424d888d11c1f949cc32_0.csv

By the end of your cell, the table should be created.  The tests are going to verify that the table exists and looks right.

In [4]:
url = 'http://dhcs-chhsagency.opendata.arcgis.com/datasets/8e4f3a0c75b9424d888d11c1f949cc32_0.csv'
data = pd.read_csv(url)

### SOLUTION GOES HERE
data.to_sql(MYTABLE, conn, index=False)
#ip_visits = pd.read_sql_query('SELECT * FROM ' + myname + '_ipv', conn)

ValueError: Table 'petersluu_data' already exists.

In [5]:
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
assert(dxyz.shape == (328,19))
assert(list(dxyz.columns) == ['Provider_Name', 'NPI', 'CCN',
       'Medicaid_EP_Hospital_Type', 'Street_Address', 'City', 'County',
       'State', 'Zip_Code', 'Payment_Year_Number', 'Program_Type',
       'Total_payments', 'Last_Program_Year', 'Last_Payment_Year',
       'Last_Payment_Criteria', 'Most_Recent_Disbursement_Amount', 'LONGITUDE',
       'LATITUDE', 'FID'])

## PART 3: Combine with other data in the database

In the database, there is an existing table called `population`.  We want to merge the DHCS datafile loaded above with the population data available in this other database table  The tables can be merged on `MYTABLE`'s `Zip_Code` field and `population`'s `zipcode` field.

Note that not all `Zip_Codes` from your downloaded file have to be in the `population` table.  If they aren't, then I want you to eliminate the non-matching records.  That is, only keep the records that have a matching ZIP code in both sets of data.

Answer the question:
Which providers are located in the zipcode with the largest population?

Put your answer in the form `answer = ['a', 'list', 'of', 'NPI', 'like', '1593042103]`

In [6]:
answer=[]

### SOLUTION GOES HERE
dhcs=pd.read_sql_query('SELECT * FROM ' + 'population', conn)
dhcs['Zip_Code']=dhcs['zipcode'].astype(int)
merged = pd.merge(data,dhcs,how='inner',on='Zip_Code')
answer.append(merged['NPI'][merged['population']==merged['population'].max()].values[0].astype(str))

In [7]:
assert(type(answer) == list)
assert(answer == ['1194016923'])

## PART 4: Total by hospital type

This is a multistep process:
* From our downloaded data file, compute the `Total payments` per ZIP code and Medicaid EP Hospital Type.
* Then merge that with the `population` data to compute a `Total payments` per person.
* Then average that across all of the `Medicaid EP Hospital Types` to get an average per persona payment for these type of hospital.

Your answer should be in structure of a data frame with at least two columns:
* Medicaid_EP_Hospital_Type
* Avg_Pay_per_Capita

In [17]:
### SOLUTION GOES HERE
answer = None
merged['Pay_Per_Pop']=merged['Total_payments']/merged['population']
answer=merged.groupby(['Zip_Code','Medicaid_EP_Hospital_Type'])['Pay_Per_Pop'].sum().reset_index()
answer=answer.rename(columns={'Pay_Per_Pop':'Avg_Pay_per_Capita'})
answer=answer.groupby('Medicaid_EP_Hospital_Type').mean()

In [18]:
assert(type(answer) == pd.core.frame.DataFrame)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Acute Care Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 17974.363)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Children\\'s Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 111.559)
